# GFK

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#adaptacion
from utils.adaptacion import gfk_grid_search, transform_gfk

#clasificadores
from utils.clasificacion import *

#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib

#variables para guardar los resultados
tipo = pruebas[2]
dataset_name = datasets[1]
dims = dimensions[dataset_name]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Pruebas con el dataset Twitter

In [3]:
print tipo
print dataset_name
print dims
print data_path

gfk
twitter
2000
data


In [4]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


# Adaptación

## Creación de modelos de adaptación.

Por cada par de dominios se entrenan distintos modelos según los parámetros establecidos.
Al entrenar cada modelo, este es inmediatamente probado, siendo almacenado o descartado según su desempeño.

Cada modelo es guardado en la ruta: models/twitter/gfk/me2\_[dominio\_fuente]_[dominio\_objetivo].pkl

### Creación de modelos de adaptación.

In [5]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_models_paths.pkl')

parameters = {
    'dims': [int(dims/4), int(dims/2)],
    'n_subs': [10, 20, 50]
}

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "%s - %s" % (src, tgt)
            X_src = np.asarray(labeled[src]['X_tr'].todense())
            y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

            X_tgt = np.asarray(labeled[tgt]['X_tr'].todense())
            
            
            gfk, score = gfk_grid_search(X_src, y_src, X_tgt, parameters, n_jobs=4)
            print "\tMejor modelo: %.3f" % score
            
            folder_path = os.path.join(models_path, dataset_name, tipo)
            prefix = "me2_" + src + "_" + tgt + ".pkl"
            
            best_model_save_path = os.path.join(folder_path, prefix)
            print "\tGuardando modelo en %s" % best_model_save_path
            joblib.dump(gfk, best_model_save_path)

print "\nAdaptaciones completadas"

Adaptando entre:
rio2016 - thevoice
	n: 10 - d: 500

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

 0.788850174216
	n: 20 - d: 500 0.866898954704
	n: 50 - d: 500 0.903275261324
	n: 10 - d: 1000 0.772682926829
	n: 20 - d: 1000 0.877212543554
	n: 50 - d: 1000 0.89574912892
	Mejor modelo: 0.903
	Guardando modelo en models/twitter/gfk/me2_rio2016_thevoice.pkl
rio2016 - general
	n: 10 - d: 500 0.796794425087
	n: 20 - d: 500 0.857003484321
	n: 50 - d: 500 0.890731707317
	n: 10 - d: 1000 0.776864111498
	n: 20 - d: 1000 0.61574912892
	n: 50 - d: 1000 0.87331010453
	Mejor modelo: 0.891
	Guardando modelo en models/twitter/gfk/me2_rio2016_general.pkl
thevoice - rio2016
	n: 10 - d: 500 0.768401543739
	n: 20 - d: 500 0.812003537543
	n: 50 - d: 500 0.868132888812
	n: 10 - d: 1000 0.775422051808
	n: 20 - d: 1000 0.800029999098
	n: 50 - d: 1000 0.8459564842
	Mejor modelo: 0.868
	Guardando modelo en models/twitter/gfk/me2_thevoice_rio2016.pkl
thevoice - general
	n: 10 - d: 500 0.801217499558
	n: 20 - d: 500 0.807696628301
	n: 50 - d: 500 0.85022292048
	n: 10 - d: 1000 0.769654233389
	n: 20 - d: 1000

### Pruebas de clasificación

In [6]:
df = pd.DataFrame(columns=dataframe_columns)

i=1
tareas = len(domains)*(len(domains)-1)

# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de %d" % (i, tareas)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #############
            #### GFK ####
            #############
            # se adaptan los dominios usando GFK
            print "Adaptando dominios..."
            folder_path = os.path.join(models_path, dataset_name, tipo)
            prefix = "me2_" + src + "_" + tgt + ".pkl"
            
            best_model_save_path = os.path.join(folder_path, prefix)
            
            gfk = joblib.load(best_model_save_path)
            
            
            X_tr = np.asarray(labeled[src]['X_tr'].todense())
            X_tr_a = transform_gfk(X_tr, gfk)
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            X_ts_a = transform_gfk(X_ts, gfk)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
            t_error = 1-clf.score(X_ts_a, y_ts)
            
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i += 1
            
print "\nPruebas completadas."

Tarea 1 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 6
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [7]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
1,GFK,r->t,rio2016,thevoice,7.362689,24.053030,16.690341
2,GFK,r->g,rio2016,general,26.998127,43.975552,16.977425
3,GFK,t->r,thevoice,rio2016,10.208711,26.270417,16.061706
4,GFK,t->g,thevoice,general,26.998127,38.746821,11.748694
5,GFK,g->r,general,rio2016,10.208711,23.321234,13.112523
6,GFK,g->t,general,thevoice,7.362689,26.089015,18.726326


In [8]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/gfk/me2_2000.csv
Resultados guardados.
